<a href="https://colab.research.google.com/github/ankit-rathi/Data-Science-with-Python/blob/master/Tradevesting/myNifty500_Prospects_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import datetime
import pytz
tz_NY = pytz.timezone('Asia/Kolkata')
datetime_NY = datetime.datetime.now(tz_NY)
print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

Run date time (IST): 2023-09-18 17:55:40


In [2]:
!pip install yfinance

In [5]:
import pandas as pd
import numpy as np
from google.colab import drive
drive.mount('/content/drive')

my500 = pd.read_csv('/content/drive/My Drive/data/stocks/myNifty500Scrips.csv')

mypf = pd.read_csv('/content/drive/My Drive/data/stocks/myProspectsScrips.csv')

myttm = pd.read_csv('/content/drive/My Drive/data/stocks/myNifty500-ATH-TTM.csv')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [25]:
#my500[my500['StarStock'] == 1]['Symbol']
tmp_df1 = my500[~my500['Symbol'].isin(mypf['Symbol'].values)]

tmp_df2 = my500[my500['StarStock'] == 1]

tmp_df3 = tmp_df2[tmp_df2['Symbol'].isin(tmp_df1['Symbol'].values)]

myss = myttm[myttm['Stock'].isin(tmp_df2['Stock'].values)]

In [27]:
myss[myss['Type'] == 'Profit']

,Type,Mar 2017,Mar 2018,Mar 2019,Mar 2020,Mar 2021,Mar 2022,Mar 2023,Latest,Stock,Max,Diff%
1,Profit,369.0,480.0,497.0,320.0,202.0,360.0,561.0,610.0,3MINDIA.NS,610.0,0.00
3,Profit,662.0,585.0,694.0,719.0,730.0,781.0,1362.0,1487.0,AIAENG.NS,1487.0,0.00
5,Profit,216.0,244.0,227.0,296.0,546.0,832.0,863.0,961.0,APLAPOLLO.NS,961.0,0.00
7,Profit,822.0,443.0,580.0,914.0,1458.0,1454.0,1865.0,2022.0,AUBANK.NS,2022.0,0.00
13,Profit,436.0,621.0,699.0,803.0,926.0,1080.0,1274.0,1389.0,ABBOTINDIA.NS,1389.0,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...
813,Profit,-12.0,13.0,35.0,-9.0,-129.0,-2.0,149.0,158.0,WESTLIFE.NS,158.0,0.00
817,Profit,11039.0,10242.0,11542.0,12252.0,13903.0,15141.0,14766.0,15211.0,WIPRO.NS,15211.0,0.00
821,Profit,300.0,382.0,411.0,223.0,152.0,196.0,427.0,474.0,ZFCVINDIA.NS,474.0,0.00
825,Profit,349.0,352.0,445.0,376.0,433.0,574.0,444.0,552.0,ZENSARTECH.NS,574.0,3.83


In [23]:
myttm

,Type,Mar 2017,Mar 2018,Mar 2019,Mar 2020,Mar 2021,Mar 2022,Mar 2023,Latest,Stock,Max,Diff%
0,Sales,2363.0,2556.0,2809.0,2765.0,2420.0,3078.0,3733.0,3821.0,3MINDIA.NS,3821.0,0.00
1,Profit,369.0,480.0,497.0,320.0,202.0,360.0,561.0,610.0,3MINDIA.NS,610.0,0.00
2,Sales,2246.0,2445.0,3070.0,2970.0,2881.0,3567.0,4909.0,5069.0,AIAENG.NS,5069.0,0.00
3,Profit,662.0,585.0,694.0,719.0,730.0,781.0,1362.0,1487.0,AIAENG.NS,1487.0,0.00
4,Sales,3924.0,5335.0,7152.0,7723.0,8500.0,13063.0,16166.0,17272.0,APLAPOLLO.NS,17272.0,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...
827,Profit,1612.0,2312.0,2382.0,1495.0,2331.0,5084.0,2585.0,3241.0,ZYDUSLIFE.NS,5084.0,36.25
828,Sales,431.0,513.0,843.0,1767.0,1867.0,2009.0,2255.0,2260.0,ZYDUSWELL.NS,2260.0,0.00
829,Profit,124.0,150.0,171.0,121.0,112.0,306.0,291.0,247.0,ZYDUSWELL.NS,306.0,19.28
830,Sales,1330.0,1365.0,1431.0,1438.0,1564.0,2160.0,2648.0,2715.0,ECLERX.NS,2715.0,0.00


In [ ]:
mypf